# Case 4 說明
對每一個 user input 執行 RAG

對每一個 input 執行 retrieve flow and related QA

假定 user 會不按理回答

In [ ]:
import os
import time
from openai import OpenAI
from tabulate import tabulate
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
import pandas as pd
import numpy as np
import faiss

csv_file = 'data\embeddingsV3_3072.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

In [ ]:
def get_similar_qa(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    question = np.array(response.data[0].embedding, ndmin=2)
    k = 3
    distances, indices = index.search(question, k)
    csv_question = 'data\manual.csv'
    qdf = pd.read_csv(csv_question)
    qarray = qdf.to_numpy()
    answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
    history = ""
    min_distance = 1
    Qcount = 0
    for i in range(0,k):
        if answer[i][1]<min_distance:
            Qcount+=1
            history = history +str(i+1)+". "+ answer[i][0][0]+"\n"+answer[i][0][1]+"\n"
    return Qcount, history

def get_similar_flow(input):
    pass

# Instruction

In [ ]:
instructions = [
    "1. You are Flora customer service chat bot.",
    "2. When user ask a question, you must follow the steps, resolving the problem one step at a time.",
    "3. If the user asks another question before the previous question is solved, deal with the new question first and then go back to the steps of the previous question for continued processing.",
]

instruction = ""
for i in instructions:
    instruction += i

print(instruction)

In [ ]:
assistant = client.beta.assistants.create(
    name="Flora's Customer Service",
    instructions=instruction,
    model="gpt-3.5-turbo-1106"
    # tools = [
    #     {
    #         "type": "function",
    #         "function": {
    #             "name": "get_history_question_and_answer",
    #             "description": "Get 1 to 3 history questions and answers that is similar to current question",
    #             "parameters": {
    #                 "type": "object",
    #                 "properties": {
    #                     "current_question": {
    #                         "type": "string",
    #                         "description": "user's question"
    #                     }
    #                 },
    #                 "required": ["current_question"]
    #             }
    #         }
    #     }
    # ]
)

thread = client.beta.threads.create()

# Flow

In [ ]:
# flow1 = [
#     "Follow the steps to answer questions:",
#     "Ask the user for their email.",
#     "Ask the user for the time when the tree was killed.",
#     "The last step, you must respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'."
# ]

# flow2 = [
#     "Follow the steps to answer questions:",
#     "Ask the user for their ID.",
#     "Ask the user for their user name.",
#     "The last step, you must respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'."
# ]

In [ ]:
'''
ask user for their email;
if(got user email){
    ask user for the time when the tree was killed;
    if(got the time){
        respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'.
    }
    else{
        ask the time again.
    }
}
else{ask email again}

ask user for their ID;
if(got user ID){
    ask user for the user name;
    if(got the user user name){
        respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'.
    }
    else{
        ask the user name again.
    }
}
else{ask ID again}
'''

flow1 = "ask user for their email;if(got user email){ask user for the time when the tree was killed;if(got the time){respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'.}else{ask the time again.}}else{ask email again}"
flow2 = "ask user for their ID;if(got user ID){ask user for the user name;if(got the user user name){respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'.}else{ask the user name again.}}else{ask ID again}"

# 2nd round

In [ ]:
# prompt = get_similar_flow(user_input) + get_similar_qa(user_input) + user_input

# prompt = ""

# for i in flow1:
#     prompt = prompt + i + "\n"

# for i in flow2:
#     prompt = prompt + i + "\n"

In [ ]:
prompt = ""

prompt += flow1

# prompt += flow2

In [ ]:
user_input = "I accidentally killed my tree. Is there any way to restore it?"
# user_input = "How do I find out which email I've linked to?"
# user_input = "abc123"
# user_input = "I just found out what email I've linked to. So how do I restore the tree?"
# user_input = "123@gmail.com"
# user_input = "It's around 4:00 PM on March 28th."

In [ ]:
similar_qa = get_similar_qa(user_input)
similar_flow = get_similar_flow(user_input)

if similar_qa[0] > 0:
    prompt += "\nHere is " + str(similar_qa[0]) + " history questions and answers related to this question according to Flora's database:\n" + similar_qa[1] + "\n"

prompt = prompt + "here is the user input \n" + user_input
print(prompt)

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content = prompt
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [ ]:
while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
print(run.status)
# print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

In [ ]:
time.sleep(5)
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order='asc'
)

In [ ]:
for i in messages.data[-3:]:
    print(i.content[0].text.value)
    print("==========")

# Delete Assistant

In [ ]:
# client.beta.assistants.delete(assistant.id)